In [ ]:
Notebook to querry GALEX field and visit information

In [26]:
from astroquery.mast import Observations
from astropy.table import Table
from astropy import units as uu
from astropy.coordinates import SkyCoord
import healpy as hpy
import numpy as np

In [ ]:
Get field info from astroquery

In [27]:
#Get TDS field info
tt_tds = Table.read("./TDS-ApJ_766_60/tds_fields_table2.ecsv")
print(tt_tds)
dd_query = {"instrument_name":"GALEX",
             "dataproduct_type":"image",
             "dataRights":"PUBLIC",
             "filters":"NUV",
             "radius":".02 deg",
             "target_name":tt_tds["name"]}
# "coordinates":tt_tds["coordinates"]

tt_obs = Observations.query_criteria(**dd_query)
print(tt_obs)
tt_obs[0:2].show_in_notebook()

      name          ra    dec   ebv  nr_vis coordinates 
                                              deg,deg   
---------------- ------- ----- ----- ------ ------------
 PS_XMMLSS_MOS00   35.58 -3.14 0.028     27  35.58,-3.14
 PS_XMMLSS_MOS01    36.5 -3.49 0.025     27   36.5,-3.49
 PS_XMMLSS_MOS02    35.0 -3.95  0.02     14   35.0,-3.95
 PS_XMMLSS_MOS03  35.875 -4.25 0.026     27 35.875,-4.25
 PS_XMMLSS_MOS04    36.9 -4.42 0.026     27   36.9,-4.42
 PS_XMMLSS_MOS05    35.2 -5.05 0.022     26   35.2,-5.05
             ...     ...   ...   ...    ...          ...
PS_VVDS22H_MOS00   333.7  1.25  0.04     39   333.7,1.25
PS_VVDS22H_MOS01   332.7   0.7 0.046     35    332.7,0.7
PS_VVDS22H_MOS02 334.428  0.67 0.057     38 334.428,0.67
PS_VVDS22H_MOS03   333.6   0.2 0.058     27    333.6,0.2
PS_VVDS22H_MOS04  334.61 -0.04 0.093     24 334.61,-0.04
PS_VVDS22H_MOS05   333.9 -0.72 0.102     35  333.9,-0.72
PS_VVDS22H_MOS06   332.9  -0.4 0.113     33   332.9,-0.4
Length = 42 rows
dataproduct_ty

idx,dataproduct_type,calib_level,obs_collection,obs_id,target_name,s_ra,s_dec,t_min,t_max,t_exptime,wavelength_region,filters,em_min,em_max,target_classification,obs_title,t_obs_release,instrument_name,proposal_pi,proposal_id,proposal_type,project,sequence_number,provenance_name,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,intentType,obsid,objID
0,image,2,GALEX,2592597978464124928,PS_XMMLSS_MOS00,35.578863351058885,-3.14176421214975,54791.262777777774,55512.31023148148,40457.25,UV,NUV,169300000000.0,300700000000.0,--,--,55953.94460633,GALEX,--,--,DIS,DIS,--,DIS,CIRCLE ICRS 35.57886335 -3.14176421 0.625,http://galex.stsci.edu/data/GR6/pipe/01-vsn/08150-PS_XMMLSS_MOS00/d/01-main/0007-img/07-try/qa/PS_XMMLSS_MOS00-xd-int_2color.jpg,http://galex.stsci.edu/data/GR6/pipe/01-vsn/08150-PS_XMMLSS_MOS00/d/01-main/0007-img/07-try/PS_XMMLSS_MOS00-nd-exp.fits.gz,PUBLIC,--,5885.0,science,2047,3659
1,image,2,GALEX,2592668347208302592,PS_XMMLSS_MOS01,36.49984981667359,-3.4923125504146,54791.33122685185,55512.37857638889,38609.0,UV,NUV,169300000000.0,300700000000.0,--,--,55953.94680547,GALEX,--,--,DIS,DIS,--,DIS,CIRCLE ICRS 36.49984982 -3.49231255 0.625,http://galex.stsci.edu/data/GR6/pipe/01-vsn/08152-PS_XMMLSS_MOS01/d/01-main/0007-img/07-try/qa/PS_XMMLSS_MOS01-xd-int_2color.jpg,http://galex.stsci.edu/data/GR6/pipe/01-vsn/08152-PS_XMMLSS_MOS01/d/01-main/0007-img/07-try/PS_XMMLSS_MOS01-nd-exp.fits.gz,PUBLIC,--,5885.0,science,2048,3661


In [37]:
tt_obs["obs_id","target_name"].pprint(100)

       obs_id         target_name   
------------------- ----------------
2592597978464124928  PS_XMMLSS_MOS00
2592668347208302592  PS_XMMLSS_MOS01
2592738715952480256  PS_XMMLSS_MOS02
2592809084696657920  PS_XMMLSS_MOS03
2592879453440835584  PS_XMMLSS_MOS04
2592949822185013248  PS_XMMLSS_MOS05
2593020190929190912  PS_XMMLSS_MOS06
2593653509626789888    PS_CDFS_MOS00
2593723878370967552    PS_CDFS_MOS01
2593794247115145216    PS_CDFS_MOS02
2593864615859322880    PS_CDFS_MOS03
2593934984603500544    PS_CDFS_MOS04
2594005353347678208    PS_CDFS_MOS05
2594075722091855872    PS_CDFS_MOS06
2597312684324028416 PS_ELAISN1_MOS10
2597383053068206080 PS_ELAISN1_MOS11
2597453421812383744 PS_ELAISN1_MOS12
2597523790556561408 PS_ELAISN1_MOS13
2597594159300739072 PS_ELAISN1_MOS14
2597664528044916736 PS_ELAISN1_MOS15
2597734896789094400 PS_ELAISN1_MOS16
2597875634277449728 PS_VVDS22H_MOS00
2597946003021627392 PS_VVDS22H_MOS01
2598016371765805056 PS_VVDS22H_MOS02
2598086740509982720 PS_VVDS22H_MOS03
2